In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from nova import Nova
from nova.types import Pose
from nova.actions import ptp, jnt

In [2]:
os.environ["CELL_ID"] = "cell"

## 🤖 Show Robot-Pad

In [3]:
from IPython.display import IFrame
IFrame(src='/cell/robot-pad', width=700, height=600)

## ⚙️ Configure Robot

In [4]:
nova = Nova()
cell = nova.cell()
controller = await cell.controller("ur")
motion_group = controller[0]

2024-12-19 12:57:12.149 | INFO     | nova.gateway:async_wrapper:31 - Calling list_controllers with args=(), kwargs={'cell': 'cell'}
2024-12-19 12:57:12.342 | INFO     | nova.gateway:async_wrapper:37 - list_controllers took 0.19 seconds


## 🔩 Get Robot Props & State

In [5]:
state = await motion_group.get_state("Flange")
state

2024-12-19 12:57:13.995 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur', 'tcp': 'Flange'}
2024-12-19 12:57:14.051 | INFO     | nova.gateway:async_wrapper:37 - get_current_motion_group_state took 0.05 seconds


MotionGroupStateResponse(state=MotionGroupState(motion_group='0@ur', controller='ur', joint_position=Joints(joints=[1.1699999570846558, -1.6579999923706055, 1.4049999713897705, -1.5709999799728394, -1.5709999799728394, 1.1699999570846558]), joint_velocity=Joints(joints=[0, 0, 0, 0, 0, 0]), joint_torque=None, flange_pose=Pose(position=Vector3d(x=-91.40618174243164, y=-662.0208072831879, z=851.325747432342), orientation=Vector3d(x=2.1397304301105744, y=2.1396749657965426, z=-0.35713579892606034), coordinate_system=''), tcp_pose=TcpPose(position=Vector3d(x=-91.40618174243164, y=-662.0208072831879, z=851.325747432342), orientation=Vector3d(x=2.1397304301105744, y=2.1396749657965426, z=-0.35713579892606034), coordinate_system='', tcp='Flange'), velocity=MotionVector(linear=Vector3d(x=0, y=0, z=0), angular=Vector3d(x=0, y=0, z=0), coordinate_system=''), force=ForceVector(force=Vector3d(x=0, y=0, z=0), moment=Vector3d(x=0, y=0, z=0), coordinate_system=''), joint_limit_reached=MotionGroupState

In [6]:
tcp_pose = await motion_group.tcp_pose("Flange")
tcp_pose

2024-12-19 12:57:15.041 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur', 'tcp': 'Flange'}
2024-12-19 12:57:15.139 | INFO     | nova.gateway:async_wrapper:37 - get_current_motion_group_state took 0.10 seconds


Pose(position=Vector3d(x=-91.40618174243164, y=-662.0208072831879, z=851.325747432342), orientation=Vector3d(x=2.1397304301105744, y=2.1396749657965426, z=-0.35713579892606034))

## 📝 Perform Pose Transforms

In [7]:
current_pose = await controller[0].tcp_pose("Flange")
# Get current TCP pose and offset it slightly along the x-axis
target_pose = current_pose @ Pose((10, 0, 0, 0, 0, 0))
print(f"Current\n{current_pose}\n\nTarget\n{target_pose}")

2024-12-19 12:57:16.320 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur', 'tcp': 'Flange'}
2024-12-19 12:57:16.392 | INFO     | nova.gateway:async_wrapper:37 - get_current_motion_group_state took 0.07 seconds


Current
(-91.4, -662.0, 851.3, 2.14, 2.14, -0.357)

Target
(-91.5, -652.3, 849.0, 2.14, 2.14, -0.357)


## 🎢 Plan a Motion

In [9]:
async with controller[0] as motion_group:
    home_joints = (await motion_group.joints("Flange")).joints
    current_pose = await motion_group.tcp_pose("Flange")
    target_pose = current_pose @ Pose((1, 0, 0, 0, 0, 0))

    actions = [
        jnt(home_joints),
        ptp(target_pose),
        jnt(home_joints),
        ptp(target_pose @ [200, 0, 0, 0, 0, 0]),
        jnt(home_joints),
        ptp(target_pose @ (300, 0, 0, 0, 0, 0)),
    ]

    plan_response = await motion_group.plan(actions, tcp="Flange")
    print(plan_response)

2024-12-19 12:57:21.999 | INFO     | nova.gateway:async_wrapper:31 - Calling activate_motion_group with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur'}
2024-12-19 12:57:22.066 | INFO     | nova.gateway:async_wrapper:37 - activate_motion_group took 0.07 seconds
2024-12-19 12:57:22.067 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur', 'tcp': 'Flange'}
2024-12-19 12:57:22.117 | INFO     | nova.gateway:async_wrapper:37 - get_current_motion_group_state took 0.05 seconds
2024-12-19 12:57:22.118 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur', 'tcp': 'Flange'}
2024-12-19 12:57:22.178 | INFO     | nova.gateway:async_wrapper:37 - get_current_motion_group_state took 0.06 seconds
2024-12-19 12:57:22.179 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=()

joint_positions=[Joints(joints=[1.1699999570846558, -1.6579999923706055, 1.4049999713897705, -1.5709999799728394, -1.5709999799728394, 1.1699999570846558]), Joints(joints=[1.1696001333426256, -1.659071656105805, 1.4062299400012528, -1.5711583329250391, -1.5708998214031384, 1.169612881784187]), Joints(joints=[1.1693667073826173, -1.6596973171406524, 1.40694802293064, -1.5712507828873643, -1.5708413466108044, 1.169386898646846]), Joints(joints=[1.1697009069586466, -1.658801548510269, 1.4059199324371816, -1.571118420839215, -1.570925065880099, 1.169710442217949]), Joints(joints=[1.169986478266023, -1.658021229018412, 1.4050179765311186, -1.5709963457333165, -1.5709966272697826, 1.1699868957647108]), Joints(joints=[1.1691766094124858, -1.6595322641410517, 1.4064700884962, -1.570919606810204, -1.5707948054162308, 1.1692023058904415]), Joints(joints=[1.1670270466286756, -1.6635348012779037, 1.4103115883954147, -1.5707110636717876, -1.5702591405553032, 1.1671198321988177]), Joints(joints=[1.1

## 🏃🏽‍♀️ Execute Motion

In [10]:
from nova.core.movement_controller import move_forward

async with controller[0] as motion_group:
    home_joints = (await motion_group.joints("Flange")).joints
    current_pose = await motion_group.tcp_pose("Flange")
    target_pose = current_pose @ Pose((1, 0, 0, 0, 0, 0))

    actions = [
        jnt(home_joints),
        ptp(target_pose),
        jnt(home_joints),
        ptp(target_pose @ [200, 0, 0, 0, 0, 0]),
        jnt(home_joints),
        ptp(target_pose @ (300, 0, 0, 0, 0, 0)),
        jnt(home_joints),
        ptp(target_pose @ Pose((300, 0, 0, 0, 0, 0))),
        jnt(home_joints),
        ptp(target_pose @ Pose((400, 0, 0, 0, 0, 0))),
        jnt(home_joints),
        ptp(target_pose),
        jnt(home_joints),
    ]

    await motion_group.run(actions, tcp="Flange", movement_controller=move_forward)

2024-12-19 12:57:23.879 | INFO     | nova.gateway:async_wrapper:31 - Calling activate_motion_group with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur'}
2024-12-19 12:57:24.104 | INFO     | nova.gateway:async_wrapper:37 - activate_motion_group took 0.22 seconds
2024-12-19 12:57:24.105 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur', 'tcp': 'Flange'}
2024-12-19 12:57:24.156 | INFO     | nova.gateway:async_wrapper:37 - get_current_motion_group_state took 0.05 seconds
2024-12-19 12:57:24.157 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=(), kwargs={'cell': 'cell', 'motion_group': '0@ur', 'tcp': 'Flange'}
2024-12-19 12:57:24.206 | INFO     | nova.gateway:async_wrapper:37 - get_current_motion_group_state took 0.05 seconds
2024-12-19 12:57:24.207 | INFO     | nova.gateway:async_wrapper:31 - Calling get_current_motion_group_state with args=()